In [1]:
import torch
from torch import nn
from torchvision import models, transforms
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [2]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Identity()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(122.7, 114.6, 100.9), std=(59.2, 58.4, 59.0)),
])


In [4]:
list_file = 'data/oxford_pets/annotations/list.txt'
root_path = 'data/oxford_pets/images/' 
labels = []
names=[]
image_paths = []

for line in open(list_file).read().splitlines():
    if line[0]!="#":
        name, class_id, species_id, breed_id = line.strip().split()
        image_paths.append(f'{root_path}/{name}.jpg')
        labels.append(int(species_id)-1)
        names.append(name)

classes = torch.tensor(labels)


In [5]:

# Batch processing parameters
batch_size = 2000  # Set batch size as needed
num_batches = len(image_paths) // batch_size

# Function to preprocess and stack images in a batch
def process_batch(batch_paths):
    images = []
    for img_path in batch_paths:
        img = Image.open(img_path).convert("RGB")  # Ensure the image is in RGB mode
        img = preprocess(img)
        images.append(img)
    return torch.stack(images)  # Create a batch of images

# Extract features in batches
features_list = []
with torch.no_grad():
    for i in range(num_batches):
        batch_paths = image_paths[i * batch_size : (i + 1) * batch_size]
        images = process_batch(batch_paths)
        batch_features = model(images)
        features_list.append(batch_features)

    # Process the remaining images
    if len(image_paths) % batch_size != 0:
        batch_paths = image_paths[num_batches * batch_size :]
        images = process_batch(batch_paths)
        batch_features = model(images)
        features_list.append(batch_features)

# Concatenate all features
features = torch.cat(features_list)

print(features.shape)

torch.Size([7349, 512])


In [6]:
scores_train, scores_valid, targets_train, targets_valid = train_test_split(
    features, classes, test_size=0.5, random_state=42, stratify=classes
)
scores_train = scores_train.numpy()
scores_valid = scores_valid.numpy()
targets_train = targets_train.numpy()
targets_valid = targets_valid.numpy()

In [31]:
logistic = LogisticRegression()
knn = KNeighborsClassifier()
kms = KMeans(n_clusters=2, random_state=75)

In [8]:
method = logistic
# Para o kmeans as classes não são necessárias
method.fit(scores_train, targets_train)
preds = method.predict(scores_valid)
(preds==targets_valid).mean()

0.7191836734693877

In [13]:
knn.fit(scores_train, targets_train)
knn_pred = knn.predict(scores_valid)
acc = accuracy_score(targets_valid, knn_pred)
acc

0.7093877551020408

In [32]:
cluster_labels_train = kms.fit_predict(scores_train)
cluster_labels_valid = kms.fit_predict(scores_valid)
(cluster_labels_valid==targets_valid).mean()


0.5066666666666667